In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from torch.utils.data import DataLoader

from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer, MBartForConditionalGeneration

from usecrets import WANDB_API_KEY
from lstm import BiLSTM
from distill import collate_fn, preprocess_function, train_distillation


from config_distill_v3_2 import (
    TEACHER_MODEL_NAME,
    MAX_SOURCE_LEN,
    MAX_TARGET_LEN,
    EMBED_DIM,
    ENC_HIDDEN_DIM,
    DEC_HIDDEN_DIM,
    NUM_LAYERS,
    DROPOUT,
    BATCH_SIZE,
    LEARNING_RATE,
    NUM_EPOCHS,
    TEMPERATURE,
    BEAM_SIZE,
    BEAM_MAX_LENGTH,
    WANDB_PROJECT,
    WANDB_RUN_NAME,
    MIN_LEN,
)


# ====================== CONFIG / CONSTANTS ======================
seed = 52
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"Running on: {torch.cuda.get_device_name()}")
    
os.environ["WANDB_API_KEY"] = WANDB_API_KEY


# ====================== MAIN ======================
def main():
    wandb.init(
        project=WANDB_PROJECT,
        name=WANDB_RUN_NAME,
        config={
            "num_train_epochs": NUM_EPOCHS,
            "batch_size": BATCH_SIZE,
            "temperature": TEMPERATURE,
            "beam_size": BEAM_SIZE,
            "teacher": TEACHER_MODEL_NAME,
            "embed_dim": EMBED_DIM,
            "ENC_HIDDEN_DIM": ENC_HIDDEN_DIM,
            "DEC_HIDDEN_DIM": DEC_HIDDEN_DIM,
            "NUM_LAYERS": NUM_LAYERS,
            "DROPOUT": DROPOUT,
            "LEARNING_RATE": LEARNING_RATE,
        },
    )

    teacher_model = MBartForConditionalGeneration.from_pretrained(
        TEACHER_MODEL_NAME
    ).to(DEVICE)
    tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL_NAME)

    tokenizer.src_lang = "ru_RU"
    tokenizer.tgt_lang = "ru_RU"

    dataset = load_dataset("json", data_files="train_smart.jsonl")["train"]
    split_data = dataset.train_test_split(test_size=0.025, seed=52)
    train_raw = split_data["train"]
    val_raw = split_data["test"]

    train_ds = train_raw.map(
        lambda x: preprocess_function(x, tokenizer, MAX_SOURCE_LEN, MAX_TARGET_LEN),
        batched=False,
    )
    val_ds = val_raw.map(
        lambda x: preprocess_function(x, tokenizer, MAX_SOURCE_LEN, MAX_TARGET_LEN),
        batched=False,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=lambda b: collate_fn(b, tokenizer.pad_token_id),
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=lambda b: collate_fn(b, tokenizer.pad_token_id),
    )

    vocab_size = len(tokenizer)
    student_model = BiLSTM(
        vocab_size=vocab_size,
        embed_dim=EMBED_DIM,
        enc_hidden_dim=ENC_HIDDEN_DIM,
        dec_hidden_dim=DEC_HIDDEN_DIM,
        pad_idx=tokenizer.pad_token_id,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT,
    ).to(DEVICE)

    train_distillation(
        teacher_model=teacher_model,
        student_model=student_model,
        train_dataloader=train_loader,
        val_dataloader=val_loader,
        tokenizer=tokenizer,
        num_epochs=NUM_EPOCHS,
        lr=LEARNING_RATE,
        temperature=TEMPERATURE,
        device=DEVICE,
        wandb_run_name=WANDB_RUN_NAME
    )

    wandb.watch(student_model, log="all")

    torch.save(
        student_model.state_dict(), f"students/{WANDB_RUN_NAME}/student_model.pt"
    )
    tokenizer.save_pretrained(f"students/{WANDB_RUN_NAME}")


if __name__ == "__main__":
    main()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Using device: cuda
Running on: NVIDIA A100 80GB PCIe


wandb: Currently logged in as: vdoninav (vdoninav-hse) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



===== Epoch 1/50 =====


Distill Loss (Train): 4.7004


...

Training (epoch 46):  73%|███████▎  | 1484/2037 [06:08<02:16,  4.05it/s, batch_loss=3.3357]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

                                                                                           

Validation -- CE: 2.7540, BERT-P: 0.6769, R: 0.6622, F1: 0.6692

===== Epoch 47/50 =====


Training (epoch 47):  73%|███████▎  | 1487/2037 [08:10<02:15,  4.06it/s, batch_loss=3.7393]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

                                                                                           

Validation -- CE: 2.7503, BERT-P: 0.6852, R: 0.6672, F1: 0.6758

===== Epoch 48/50 =====


Training (epoch 48):  73%|███████▎  | 1487/2037 [10:35<03:53,  2.35it/s, batch_loss=3.6604]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Training (epoch 49):  73%|███████▎  | 1493/2037 [10:38<03:53,  2.33it/s, batch_loss=3.4050]